In [1]:
import seaborn as sns
import networkx
import folium
import pandas as pd
import osmnx as ox
import ipywidgets as widgets
ox.config(use_cache=True, log_console=True)

2022-02-11 20:35:49 Configured OSMnx 1.1.2
2022-02-11 20:35:49 HTTP response caching is on


In [2]:
!jupyter serverextension enable voila
!jupyter server extension enable voila

Enabling: voila
- Writing config: /Users/witoldtenhove/.jupyter
    - Validating...
      voila 0.3.1 OK
Enabling: voila
- Writing config: /Users/witoldtenhove/opt/anaconda3/envs/osmnx/etc/jupyter
    - Validating voila...
      voila 0.3.1 OK
    - Extension successfully enabled.


## Load data

In [3]:
url = 'routes_data.csv'
routes_df = pd.read_csv(url)
routes_df.head()

,route_id,arrival,activity_id,shift_id,longitude,latitude
0,0,1313.393,77.0,6.0,6.164641,52.269232
1,0,1313.393,68.0,5.0,6.164916,52.269976
2,0,1381.537,1.0,0.0,6.164124,52.269695
3,0,1440.034,50.0,3.0,6.172841,52.265654
4,0,2281.557,79.0,6.0,6.172033,52.266394


## Create base map

In [4]:
base_map = folium.Map(location=[routes_df.latitude.mean(
), routes_df.longitude.mean()], zoom_start=15, control_scale=True)
base_map

## Plot all client locations by route (colors)

In [5]:
n_colors = len(pd.unique(routes_df.route_id.values))
colors = sns.color_palette("Set2", n_colors).as_hex()

m = base_map
for index, location_info in routes_df.iterrows():
    folium.Circle(
        [location_info["latitude"],
         location_info["longitude"]],
        radius=30,
        color="DimGray",
        fill_color=colors[int(location_info["route_id"])],
        fill=True,
        fill_opacity=0.85,
        popup=location_info["route_id"]
    ).add_to(m)
m

## Trace routes on map

In [6]:
deventer_graph = ox.load_graphml("deventer_graph.graphml")

## Function to trace a single route (by route_id) on map

def calc_route(route_id: int, path_coordinates, graph):
    route_id = route_id
    # Get arrays of longitudes and latitudes for path for a given rout_id
    lon_array = path_coordinates[path_coordinates.route_id ==
                                 route_id].longitude
    lat_array = path_coordinates[path_coordinates.route_id ==
                                 route_id].latitude

    # Calculates nearest nodes for all path coordinates from arrays
    node_list = ox.distance.nearest_nodes(
        G=deventer_graph,
        X=lon_array,
        Y=lat_array
    )

    # for each pair of subsequent nodes calculate shortest path and append array of path nodes to route array
    route = []
    for i in range(len(node_list) - 1):
        start = node_list[i]
        end = node_list[i + 1]
        path_nodes = networkx.shortest_path(graph, start, end)
        route.append(path_nodes)
        i += 1

    # return route as list
    return(route)

2022-02-11 20:35:52 Converting node, edge, and graph-level attribute data types
2022-02-11 20:35:53 Loaded graph with 20870 nodes and 39886 edges from "deventer_graph.graphml"


## Plot route on map

In [7]:
route_id = 5

route_list = calc_route(route_id, routes_df, deventer_graph)
route_df_flr = routes_df[routes_df['route_id'] == route_id]
route_map = folium.Map(location=[route_df_flr.latitude.mean(
), route_df_flr.longitude.mean()], control_scale=True)
for index, location_info in route_df_flr.iterrows():
    folium.Circle(
        [location_info["latitude"],
         location_info["longitude"]],
        radius=20,
        color="DimGray",
        fill_color=colors[int(location_info["route_id"])],
        fill=True,
        fill_opacity=0.85,
        popup=round(location_info["arrival"])
    ).add_to(route_map)

for i, stage in enumerate(route_list):
    route_map = ox.folium.plot_route_folium(
        G=deventer_graph, route=stage, route_map=route_map, color='red', weight=2)

sw = route_df_flr[['latitude', 'longitude']].min().values.tolist()
ne = route_df_flr[['latitude', 'longitude']].max().values.tolist()
route_map.fit_bounds([sw, ne])
route_map



2022-02-11 20:35:53 Created nodes GeoDataFrame from graph
2022-02-11 20:35:53 Created edges GeoDataFrame from graph
2022-02-11 20:35:53 Created edges GeoDataFrame from graph
2022-02-11 20:35:53 Created edges GeoDataFrame from graph
2022-02-11 20:35:53 Created edges GeoDataFrame from graph
2022-02-11 20:35:53 Created edges GeoDataFrame from graph
2022-02-11 20:35:53 Created edges GeoDataFrame from graph
2022-02-11 20:35:53 Created edges GeoDataFrame from graph
2022-02-11 20:35:53 Created edges GeoDataFrame from graph
2022-02-11 20:35:53 Created edges GeoDataFrame from graph
2022-02-11 20:35:53 Created edges GeoDataFrame from graph
2022-02-11 20:35:53 Created edges GeoDataFrame from graph
2022-02-11 20:35:53 Created edges GeoDataFrame from graph
2022-02-11 20:35:53 Created edges GeoDataFrame from graph
2022-02-11 20:35:53 Created edges GeoDataFrame from graph
2022-02-11 20:35:53 Created edges GeoDataFrame from graph


In [8]:

def trace_route(route_id):
    route_list = calc_route(route_id, routes_df, deventer_graph)
    route_df_flr = routes_df[routes_df['route_id'] == route_id]
    route_map = folium.Map(location=[route_df_flr.latitude.mean(
    ), route_df_flr.longitude.mean()], control_scale=True)
    for index, location_info in route_df_flr.iterrows():
        folium.Circle(
            [location_info["latitude"],
             location_info["longitude"]],
            radius=20,
            color="DimGray",
            fill_color=colors[int(location_info["route_id"])],
            fill=True,
            fill_opacity=0.85,
            popup=round(location_info["arrival"])
        ).add_to(route_map)

    for i, stage in enumerate(route_list):
        route_map = ox.folium.plot_route_folium(
            G=deventer_graph, route=stage, route_map=route_map, color='red', weight=2)

    sw = route_df_flr[['latitude', 'longitude']].min().values.tolist()
    ne = route_df_flr[['latitude', 'longitude']].max().values.tolist()
    route_map.fit_bounds([sw, ne])
    display(route_map)

In [9]:
routeWidget = widgets.RadioButtons(
   options=[0, 1, 2],
   description='RouteID:',
   disabled=False
)
    
widgets.interactive(trace_route, route_id=routeWidget)



interactive(children=(RadioButtons(description='RouteID:', options=(0, 1, 2), value=0), Output()), _dom_classe…